In [ ]:
from src import data_utils

In [ ]:
# Prepare dataset
with open("data/raw_dataset.txt", "r") as f_in:
    dataset = [line for line in f_in]
    cleaned = list(map(data_utils.clean_string, dataset))
    data_utils.split(
        cleaned,
        "data/train.txt",
        "data/val.txt",
        "data/test.txt")

    # with open("data/clean_dataset.txt", "w") as f_out:
    #     f_out.writelines(cleaned)